In [ ]:
from __future__ import print_function

import os
import numpy as np

import cv2




image_rows = 256
image_cols = 256
skip = 1

def read_imgs(img_path, mask_path):

    images = os.listdir(img_path) # 파일이름 저장 리스트
    total = len(images)  # 파일갯수

    mask_images = os.listdir(mask_path)  # 마스크
    mask_total = len(mask_images)

    imgs = np.zeros((total//skip, image_rows, image_cols), dtype=np.uint8)
    mask_imgs = np.zeros((mask_total//skip, image_rows, image_cols), dtype=np.uint8)

    if(total != mask_total):
        print ('Error in number of images and mask_images')
        return imgs,mask_imgs

    i = 0
    print('Making numpy array from images...')
    for image_name in images:
        if i % skip == 0 and i//skip < total//skip :
            print (i, i//skip)
            img = cv2.imread(os.path.join(img_path, image_name), cv2.IMREAD_GRAYSCALE)
            tmp = cv2.resize(img, (image_cols, image_rows), interpolation=cv2.INTER_CUBIC)
            imgs[i//skip] = tmp

            img = cv2.imread(os.path.join(mask_path, image_name), cv2.IMREAD_GRAYSCALE)
            tmp = cv2.resize(img, (image_cols, image_rows), interpolation=cv2.INTER_CUBIC)
            mask_imgs[i//skip] = tmp
        i += 1
    print(str(total//skip) + '  Loading done.')

    return imgs, mask_imgs

from google.colab.patches import cv2_imshow

def vis_img_mask(imgs, mask_imgs, wn):

    size = int(imgs.shape[0]/5)
    w = imgs.shape[2]
    h = imgs.shape[1]
    t_img = np.zeros((2*h,6*w), np.uint8)
    
    for i in range (5):
        t_img[0:h,i*w:i*w+w] = imgs[i*size]
        t_img[h:h+h,i*w:i*w+w] = mask_imgs[i*size]
    t_img[0:h,5*w:5*w+w] = imgs[imgs.shape[0]-1]
    t_img[h:h+h,5*w:5*w+w] = mask_imgs[imgs.shape[0]-1]

    cv2_imshow(t_img)


data_dir = 'drive/MyDrive/python/data/raw-cmr-small/'

def create_train_data():

    imgs, mask_imgs = read_imgs(data_dir+'train', data_dir+'train_mask')
    np.save('imgs_train.npy', imgs)
    np.save('imgs_mask_train.npy', mask_imgs)

    vis_img_mask(imgs, mask_imgs, 'train')


def load_train_data():
    imgs_train = np.load('imgs_train.npy')
    imgs_mask_train = np.load('imgs_mask_train.npy')
    return imgs_train, imgs_mask_train


def create_test_data():

    imgs, mask_imgs = read_imgs(data_dir+'test', data_dir+'test_mask')
    np.save('imgs_test.npy', imgs)
    np.save('imgs_mask_test.npy', mask_imgs)

    vis_img_mask(imgs, mask_imgs, 'test')


def load_test_data():
    imgs_test = np.load('imgs_test.npy')
    imgs_mask_test = np.load('imgs_mask_test.npy')
    return imgs_test, imgs_mask_test

if __name__ == '__main__':
    create_train_data()
    create_test_data()

In [ ]:
"""[ 구글 colab 에서의 unet train.py 예제 프로그램 ]


---


"""

# npy 영상을 읽어 (img_rows, img_cols) 크기로 영상을 resize 하고 위 keras data 형식으로 
# 누적하여 만들어줌.  
# 영상을 255로 나누어 level을 정규화 하고 u-net 에 입력하여 train 하고 test 함.

from __future__ import print_function
import cv2
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Dropout, Conv2D, Conv3D
from keras.layers import MaxPooling2D, MaxPooling3D, Conv2DTranspose, UpSampling2D
from keras.layers import BatchNormalization, Activation
from keras.layers import Reshape, Dense, multiply, Add, Concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from keras import models
from keras import callbacks

img_rows = 256  # u-net 입력영상 높이
img_cols = 256   # u-net 입력영상 너비
ch = 3 ##수정## 4->3

smooth = 1.

#data_path = 'drive/My Drive/raw-cmr-small/'
data_path = './'

import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

def load_train_data():
    imgs_train = np.load(data_path+'imgs_train.npy')
    imgs_mask_train = np.load(data_path+'imgs_mask_train.npy')
    return imgs_train, imgs_mask_train

def load_test_data():
    imgs_test = np.load(data_path+'imgs_test.npy')
    imgs_mask_test = np.load(data_path+'imgs_mask_test.npy')
    return imgs_test, imgs_mask_test

# 테스트 결과를 dice_coef로 평가, 완전히 겹치면 1로 100%, 0 이면 0%
# 테스트 결과는 테스트 영상으로 확인
# y_true : 테스트 영상의 나와야 할 결과 
# y_pred : 테스트 영상을 입력했을 때 u-net 결과



def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


# def preprocess(imgs):

#     # 4D keras 데이타 형식의 numpy 배열 만들기, imgs.shape[0]가 영상 개수
#     # imgs 는 data.py에서 만들어 저장한 파일
#     imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols, ch), dtype=np.uint8)

#     for i in range(imgs.shape[0]):
#         tmp1 = cv2.resize(imgs[i], (img_cols, img_rows), 
#                     interpolation=cv2.INTER_CUBIC)
#         # tmp1은 (img_rows, img_cols) 크기의 2D 영상. 
#         # 이 tmp1을 (imgs.shape[0], img_rows, img_cols, 1) 4D 배열에 넣기
#         imgs_p[i,:,:,0] = tmp1  # ch이 1 이상인 경우 의미에 맞게 추가
#     return imgs_p

def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols, ch), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        tmp1 = cv2.resize(imgs[i], (img_cols, img_rows), interpolation=cv2.INTER_CUBIC)
        imgs_p[i,:,:,0] = tmp1
        imgs_p[i,:,:,1] = tmp1
        imgs_p[i,:,:,2] = tmp1

    return imgs_p

# def preprocess_mask(imgs):
#     imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols, 1), dtype=np.uint8)
#     for i in range(imgs.shape[0]):
#         tmp = cv2.resize(imgs[i], (img_cols, img_rows), 
#                          interpolation=cv2.INTER_CUBIC)
#         imgs_p[i,:,:,0] = tmp
#         #imgs_p[i,:,:,1] = 255-tmp
#     return imgs_p

def preprocess_mask(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols, 2), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        tmp = cv2.resize(imgs[i], (img_cols, img_rows), interpolation=cv2.INTER_CUBIC)
        imgs_p[i,:,:,0] = tmp
        imgs_p[i,:,:,1] = 255-tmp
    return imgs_p


def train_and_predict():
    print('Loading and preprocessing train data...')
    imgs_train, imgs_mask_train = load_train_data()

    imgs_train = preprocess(imgs_train)
    imgs_mask_train = preprocess_mask(imgs_mask_train)

    t_img = imgs_train[0, :, :, 0]
    
    from google.colab.patches import cv2_imshow
    cv2_imshow(t_img)
    #cv2.imshow("img", t_img)
    #cv2.moveWindow("img", 20, 20);
    #cv2.waitKey(100)

    t_img = imgs_mask_train[0, :, :, 0]
    cv2_imshow(t_img)
    #cv2.imshow("mask", t_img)
    #cv2.moveWindow("mask", 20, 320);
    #cv2.waitKey(100)

    # 영상 level 0~1로 정규화
    imgs_train = imgs_train.astype('float32') / 255
    imgs_mask_train = imgs_mask_train.astype('float32') / 255

##수정##
    # validation data set : hyperparameter
    # np.random.seed(1234)
    # valid_size = int(0.2*imgs_train.shape[0])
    # random_index = np.random.choice(imgs_train.shape[0], imgs_train.shape[0], 
    #                  replace=False) # 100개인 경우 100개를 random하게 섞는다.
    # valid_imgs_train = imgs_train[random_index[:valid_size]] # 0~20까지
    # valid_imgs_mask_train = imgs_mask_train[random_index[:valid_size]]
    # imgs_train = imgs_train[random_index[valid_size:]]  # 20~100까지
    # imgs_mask_train = imgs_mask_train[random_index[valid_size:]]

    print('Creating and compiling model...')

    # model = get_unet2()
    # model.summary()

    # model_checkpoint=ModelCheckpoint('Deeplabv3.hdf5',monitor='loss',save_best_only=True) ##수정##

    from model import Deeplabv3

    model = Deeplabv3(weights=None,input_tensor=None,input_shape=(img_rows, img_cols, ch),
                      classes=2, backbone='xception',OS=16,alpha=1.,activation='softmax')
    
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy']) 
    model.summary()
    

    # batch_size는 GPU에서 일괄 처리 단위. GPU 메모리에 맞게 최대한 크게 설정하는 것이
    # 속도가 빠름. 특별한 이유없이 CPU memory alloc error가 나면 컴퓨터 재부팅 후 시도
    
    print('Fitting model...')
    epoch = 10
    history = model.fit(imgs_train, imgs_mask_train, batch_size=2, epochs=epoch,
                        verbose=1, shuffle=True)
                        # validation_data=(valid_imgs_train, valid_imgs_mask_train), 
                        # callbacks=[model_checkpoint]) ##수정##

    history_dict = history.history
    print(history_dict)
    loss_values = history_dict['loss']
    # val_loss_values = history_dict['val_loss'] ##수정##
    print(loss_values)
    # print(val_loss_values)

    epochs = range(1, epoch + 1)

    from matplotlib import pyplot as plt
    plt.plot(epochs, loss_values, 'b+', label='Training loss')
    # plt.plot(epochs, val_loss_values, 'g+', label='Validation loss') ##수정##
    plt.title('Training')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()


    # test 영상에 대해 segmentation 하기
    print('Loading and preprocessing test data...')
    imgs_test, imags_mask_test = load_test_data()
    imgs_test = preprocess(imgs_test)
    imags_mask_test = preprocess(imags_mask_test)

    imgs_test = imgs_test.astype('float32') / 255
    imags_mask_test = imags_mask_test.astype('float32') / 255

    print('Loading saved weights...')
    # 이부분 decode() 오류 나면 다음 설치 : pip install h5py==2.9.0
    !pip install h5py==2.9.0
    model.load_weights('unet.hdf5')

    print('Predicting masks on test data...')
    imgs_test_pred = model.predict(imgs_test, batch_size=2, verbose=1)
    np.save('imgs_test_pred.npy', imgs_test_pred) ##수정## data_path 삭제

    # dice coef로 성능 구하기
##수정##
    # smooth = 1.
    # dice_coeff = 0.0
    # for i in range(imgs_test.shape[0]):
    #     y_true_f = imags_mask_test[i,:,:,0]
    #     y_pred_f = imgs_test_pred[i,:,:,0]

    smooth = 1.
    dice_coeff = 0.0
    for i in range(imgs_test.shape[0]):
        y_true_f = cv2.threshold(imags_mask_test[i,:,:,0], 0.5,1., cv2.THRESH_BINARY)[1].astype(np.uint8)
        y_pred_f = cv2.threshold(imgs_test_pred[i,:,:,0], 0.5, 1., cv2.THRESH_BINARY)[1].astype(np.uint8)

        intersection = np.sum(y_true_f * y_pred_f)
        dice = (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)
        dice_coeff = dice_coeff + dice
        print (i, dice)
    dice_coeff = dice_coeff / imgs_test.shape[0]
    print (dice_coeff)

    # groud_truth(mask)와 결과값(imgs_test_pred) sample display
    # 전체 중 5개 만 display
    size = imags_mask_test.shape[0]//5
    w = imags_mask_test.shape[2]
    h = imags_mask_test.shape[1]
    t_img = np.zeros((2*h,5*w), np.uint8)
    for i in range (5):
        t_img[0:h,i*w:i*w+w] = 255*imags_mask_test[i*size][:,:,0]
        t_img[h:h+h,i*w:i*w+w] = 255*imgs_test_pred[i*size][:,:,0]

    cv2.putText(t_img, 'ground-truth', (0,20), 2, 1, (192, 192, 192))
    cv2.putText(t_img, 'result', (0,256+20), 2, 1, (192, 192, 192))
    cv2_imshow(t_img)
    #cv2.imshow('mask', t_img)
    #cv2.moveWindow('mask', 20,0);
    #cv2.waitKey(0)

if __name__ == '__main__':
    train_and_predict()

In [ ]:
"""[ 구글 colab 에서의 unet color_view.py 예제 프로그램 ]

"""

from __future__ import print_function

import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np

img_cols = 256
img_rows = 256

#data_path = 'drive/My Drive/raw-cmr-small/‘
data_path = './'

def prep(img):
    img = img.astype('float32')
    img = cv2.threshold(img, 0.7, 1., cv2.THRESH_BINARY)[1]
    img = cv2.resize(img, (img_cols, img_rows))
    return img

def set_view():
    imgs_test = np.load(data_path+'imgs_test.npy')
    imgs_test_pred = np.load(data_path+'imgs_test_pred.npy')
    total=imgs_test.shape[0]

    print (imgs_test_pred.shape)

    dt_img = np.ndarray((total, img_rows, img_cols), dtype=np.uint8)
    rgb_img = np.ndarray((total, img_rows, img_cols, 3), dtype=np.uint8)

    for m in range(total):

        print (m)

        gray_img = imgs_test[m]
        gray_img = cv2.resize(gray_img, (img_cols, img_rows))

        t_img = imgs_test_pred[m,:,:,0]
        t_img = cv2.resize(t_img, (img_cols, img_rows))
        dt_img[m] = 255*t_img

        bin_img = prep(t_img)

        kernel = np.ones((3,3), np.uint8)
        dilated_img = cv2.dilate(128*bin_img, kernel, iterations=1)

        boundary_img = dilated_img * (1-bin_img)

        rgb_img[m][:,:,0] = gray_img
        rgb_img[m][:,:,1] = np.clip(gray_img + boundary_img, 0, 255)
        rgb_img[m][:,:,2] = gray_img

    return rgb_img, dt_img


view_imgs, dt_imgs = set_view()

imgs_test_pred = np.load(data_path+'imgs_test_pred.npy')
d_cnt = imgs_test_pred.shape[0]

fig, ax = plt.subplots(d_cnt, 2, figsize=(20,60))

for m in range(imgs_test_pred.shape[0]):
    plt.subplot(d_cnt, 2, 2*m+1)   # 행의갯수, 열의갯수, 위치
    plt.imshow(dt_imgs[m], cmap="gray", vmax=255)
    plt.subplot(d_cnt, 2, 2*m+2)   # 행의갯수, 열의갯수, 위치
    plt.imshow(view_imgs[m])

plt.show()